In [27]:
# 플레이어가 해야할 행동
# 1 전사를 피하기
# 2 최적의 시야 방햐을 선택하여 전사를 제거
# 최종적으로 종료 지점에 도달

### BFS를 이용하여 최단 거리 계산

import sys
from collections import deque
from typing import List, Tuple


inf = int(1e9) + 10  # BFS 거리 초기화를 위한 무한대 값 생성
x_direction = [-1, 1, 0, 0]   # 네 방향 탐색 (위, 아래, 왼쪽, 오른쪽)
y_direction = [0, 0, -1, 1]
point = Tuple[int, int]  # 좌표 표현을 위한 타입 힌트

def moving_distance(a:point,b:point)->int : ## 격자 형태에서의 이동, 위, 아래, 좌, 우로의 이동 가능 -> 대각선 이동 불가

  return abs(a[0]-b[0]) + abs(a[1]-b[1]) ##abs(a[0]-b[0])-> 가로의 이동거리 ( x좌표 끼리의 차이 ) + abs(a[1]-b[1])-> 세로의 이동거리 ( y좌표 끼리의 차이 )


## 격자형 맵에서 모든 셀까지의 최단 거리를 한번의 탐색으로 구할수 있음
## 모든 경로의 가중치가 동일, 한번 방문한 곳은 다시 방문하지 않음
## 특정 위치에서 시작하여 모든 도달 가능한 위치 까지의 최단 거리를 BFS로 계산

def cal_distance(ini_x:int, ini_y:int, N:int, obstacle:List[List[int]]) -> List[List[int]]: ## obstacle이랑, 결과(최단거리 정보 저장)는 행렬의 형태로 나타나기 때문에, List[List[int]]의 형식
  ## 거리 배열 -> 장애물이 있으면 inf 값, 나머지는 -1
  distance_map = [[inf if obstacle[i][j] else -1 for j in range(N)] for i in range(N)]
  distance_map[ini_x][ini_y] = 0

  q = deque() ## BFS를 수행할 queue 생성
  q.append((ini_x, ini_y))

  ## BFS 실행
  while q:
    x, y = q.popleft() ## queue에서 현재 탐색할 위치를 꺼내고, 상, 하, 좌, 우로 이동 시도

    for dir in range(4):
      nx = x + x_direction[dir]
      ny = y + y_direction[dir]

      if nx < 0 or nx >= N or ny < 0 or ny >= N: ## 범위 밖인 경우
        continue

      if distance_map[nx][ny] != -1: ## 이미 방문한 경우
        continue

      distance_map[nx][ny] = distance_map[x][y] + 1 ## 새로운 위치의 최단거리 갱신
      q.append((nx, ny)) ## 큐에는 새로운 위치를 추가 BFS 탐색 계속 진행

  return distance_map ## BFS 탐색이 완료된 이후에 최단거리 정보가 담겨 있는 distance_map (행렬형식)을 반환

## 플레이어가 위쪽을 바라볼 때의 시야 계산
## 플레이어의 시야 내에 있는 전사를 감지, 시야를 설정
## 단순한 직선 시야가 아니라 좌우로 퍼지는 다이아 몬드 형태(90도 각도)로 시야 설정
## 이동 거리가 증가할수록 시야 폭 역시 증가
## 맵의 경계를 벗어나지 않도록 left, right 설정


def sight_up(x:int, y:int, N:int, warrior:List[List[int]], sight_map:List[List[int]]) -> int:


  for i in range(x-1, -1, -1): ## 위쪽 방향의 탐색 ( i=x-1이면 플레이어가 위치한 바로 윗행, i = 0이면, 격자의 맨 윗부분)
    left = max(0,y - (x-i)) ## 왼쪽 방향의 한계 (x-i는 플레이어와 현재 줄 i 사이의 거리 여기서 y값을 빼면, 좌측으로 대칭 확장되는 다이아 몬드 형태를 생성) , 0보다 작아질수 없도록 제한
    right = min(N-1, y + (x-i)) ## 오른쪽 방향의 한계  (x-i는 플레이어와 현재 줄 i 사이의 거리 여기서 y값을 더하면, 우측으로 대칭 확장되는 다이아 몬드 형태를 생성), 그리드 밖으로 나가지 않도록 제한

    for j in range(left, right+1): ## 왼쪽 방향의 한계 부터 오른쪽 방향의 한계 까지 시야를 설정
      sight_map[i][j] = 1

  ## 장애물 처리를 통해 시야의 막힘 여부 확인
  obstruction_found = False  ## 초기에는 장애물이 없는 상태
  for i in range(x - 1, -1, -1):
    if obstruction_found:
        sight_map[i][y] = 0  # 장애물이 발견된 후에는 시야 차단
    else:
        sight_map[i][y] = 1  # 장애물이 발견되지 않으면 시야 유지

    if warrior[i][y]:
        obstruction_found = True  # 전사가 있으면 이후 시야 차단

## 시야 내의 전사 수를 계산

  coverage = 0
  for i in range(x - 1, -1, -1):
    left = max(0, y - (x - i))
    right = min(N - 1, y + (x - i))
    for j in range(left, right + 1):
      if sight_map[i][j]:
        coverage += warrior[i][j] ## 위쪽 탐색하면서 sight 맵에서 전사수 더함 -> 최종적으로 시야 내에서 제거할 수 있는 전사 수 반환

  return coverage

def sight_down(x: int, y: int, N: int, warrior: List[List[int]],
              sight_map: List[List[int]]) -> int:

    # 다이아몬드 형태로 아래쪽 셀을 시야에 포함시킴
    for i in range(x + 1, N):
        left = max(0, y - (i - x))
        right = min(N - 1, y + (i - x))
        for j in range(left, right + 1):
            sight_map[i][j] = 1  # 시야 설정

    # 장애물 처리: 시야 막힘 여부 확인
    obstruction_found = False
    for i in range(x + 1, N):
        if obstruction_found:
            sight_map[i][y] = 0  # 장애물이 발견된 후에는 시야 제거
        else:
            sight_map[i][y] = 1  # 장애물이 발견되지 않으면 시야 유지

        if warrior[i][y]:
            obstruction_found = True  # 전사가 있는 경우 장애물로 간주

    # 장애물에 따라 시야 조정
    for i in range(x + 1, N - 1):
        left = max(0, y - (i - x))
        right = min(N - 1, y + (i - x))

        # 왼쪽 측면 조정
        for j in range(left, y):
            if not sight_map[i][j] or warrior[i][j]:
                if j > 0:
                    sight_map[i + 1][j - 1] = 0  # 왼쪽 아래 셀의 시야 제거
                sight_map[i + 1][j] = 0          # 바로 아래 셀의 시야 제거

        # 오른쪽 측면 조정
        for j in range(y + 1, right + 1):
            if not sight_map[i][j] or warrior[i][j]:
                if j + 1 < N:
                    sight_map[i + 1][j + 1] = 0  # 오른쪽 아래 셀의 시야 제거
                sight_map[i + 1][j] = 0          # 바로 아래 셀의 시야 제거

    # 시야로 커버된 전사 수 계산
    coverage = 0
    for i in range(x + 1, N):
        left = max(0, y - (i - x))
        right = min(N - 1, y + (i - x))
        for j in range(left, right + 1):
            if sight_map[i][j]:
                coverage += warrior[i][j]

    return coverage  # 커버리지 반환


def sight_left(x: int, y: int, N: int, warrior: List[List[int]],
              sight_map: List[List[int]]) -> int:

    # 다이아몬드 형태로 왼쪽 셀을 시야에 포함시킴
    for i in range(y - 1, -1, -1):
        top = max(0, x - (y - i))
        bottom = min(N - 1, x + (y - i))
        for j in range(top, bottom + 1):
            sight_map[j][i] = 1  # 시야 설정

    # 장애물 처리: 시야 막힘 여부 확인
    obstruction_found = False
    for i in range(y - 1, -1, -1):
        if obstruction_found:
            sight_map[x][i] = 0  # 장애물이 발견된 후에는 시야 제거
        else:
            sight_map[x][i] = 1  # 장애물이 발견되지 않으면 시야 유지

        if warrior[x][i]:
            obstruction_found = True  # 전사가 있는 경우 장애물로 간주

    # 장애물에 따라 시야 조정
    for i in range(y - 1, 0, -1):
        top = max(0, x - (y - i))
        bottom = min(N - 1, x + (y - i))

        # 상단 측면 조정
        for j in range(top, x):
            if not sight_map[j][i] or warrior[j][i]:
                if j > 0:
                    sight_map[j - 1][i - 1] = 0  # 왼쪽 위 셀의 시야 제거
                sight_map[j][i - 1] = 0          # 바로 왼쪽 셀의 시야 제거

        # 하단 측면 조정
        for j in range(x + 1, bottom + 1):
            if not sight_map[j][i] or warrior[j][i]:
                if j + 1 < N:
                    sight_map[j + 1][i - 1] = 0  # 왼쪽 아래 셀의 시야 제거
                sight_map[j][i - 1] = 0          # 바로 왼쪽 셀의 시야 제거

    # 시야로 커버된 전사 수 계산
    coverage = 0
    for i in range(y - 1, -1, -1):
        top = max(0, x - (y - i))
        bottom = min(N - 1, x + (y - i))
        for j in range(top, bottom + 1):
            if sight_map[j][i]:
                coverage += warrior[j][i]

    return coverage  # 커버리지 반환


def sight_right(x: int, y: int, N: int, warrior: List[List[int]],
               sight_map: List[List[int]]) -> int:

    # 다이아몬드 형태로 오른쪽 셀을 시야에 포함시킴
    for i in range(y + 1, N):
        top = max(0, x - (i - y))
        bottom = min(N - 1, x + (i - y))
        for j in range(top, bottom + 1):
            sight_map[j][i] = 1  # 시야 설정

    # 장애물 처리: 시야 막힘 여부 확인
    obstruction_found = False
    for i in range(y + 1, N):
        if obstruction_found:
            sight_map[x][i] = 0  # 장애물이 발견된 후에는 시야 제거
        else:
            sight_map[x][i] = 1  # 장애물이 발견되지 않으면 시야 유지

        if warrior[x][i]:
            obstruction_found = True  # 전사가 있는 경우 장애물로 간주

    # 장애물에 따라 시야 조정
    for i in range(y + 1, N - 1):
        top = max(0, x - (i - y))
        bottom = min(N - 1, x + (i - y))

        # 상단 측면 조정
        for j in range(top, x):
            if not sight_map[j][i] or warrior[j][i]:
                if j > 0:
                    sight_map[j - 1][i + 1] = 0  # 오른쪽 위 셀의 시야 제거
                sight_map[j][i + 1] = 0          # 바로 오른쪽 셀의 시야 제거

        # 하단 측면 조정
        for j in range(x + 1, bottom + 1):
            if not sight_map[j][i] or warrior[j][i]:
                if j + 1 < N:
                    sight_map[j + 1][i + 1] = 0  # 오른쪽 아래 셀의 시야 제거
                sight_map[j][i + 1] = 0          # 바로 오른쪽 셀의 시야 제거

    # 시야로 커버된 전사 수 계산
    coverage = 0
    for i in range(y + 1, N):
        top = max(0, x - (i - y))
        bottom = min(N - 1, x + (i - y))
        for j in range(top, bottom + 1):
            if sight_map[j][i]:
                coverage += warrior[j][i]

    return coverage  # 커버리지 반환


def choose_best_sight(x: int, y: int, N: int, warrior: List[List[int]],
                      sight_map: List[List[int]]) -> int:

    # 시야 맵 초기화 (모든 셀 0으로)
    for i in range(N):
        for j in range(N):
            sight_map[i][j] = 0

    max_coverage = -1  # 최대 커버리지를 저장할 변수
    best_direction = -1  # 최적의 시야 방향 (0: 위, 1: 아래, 2: 왼쪽, 3: 오른쪽)

    # 시야 방향별 함수 리스트 (순서: 위, 아래, 왼쪽, 오른쪽)
    sight_functions = [sight_up, sight_down, sight_left, sight_right]

    # 각 방향별로 시야를 설정하고 커버리지를 계산
    for d in range(4):
        # 매 방향마다 sight_map을 0으로 초기화
        for i in range(N):
            for j in range(N):
                sight_map[i][j] = 0

        # 해당 방향으로 시야를 설정하고, 커버리지를 계산
        coverage = sight_functions[d](x, y, N, warrior, sight_map)
        if coverage > max_coverage:
            max_coverage = coverage
            best_direction = d

   # 최적의 방향으로 최종 시야를 설정하기 위해 sight_map 초기화
    for i in range(N):
        for j in range(N):
            sight_map[i][j] = 0

    if best_direction != -1:
        sight_functions[best_direction](x, y, N, warrior, sight_map)

    return max_coverage


def move_warriors(player_x: int, player_y: int, N: int, M: int, warrior_positions: List[point],
                  sight_map: List[List[int]]) -> Tuple[int, int]:
    total_moved = 0  # 총 이동한 전사 수
    total_hits = 0   # 플레이어에게 도달한 전사 수

    # 전사의 이동 방향: 위, 아래, 왼쪽, 오른쪽
    move_dx = [-1, 1, 0, 0]
    move_dy = [0, 0, -1, 1]

    # 모든 전사에 대해 이동 처리
    for i in range(M):
        if warrior_positions[i][0] == -1:
            continue  # 이미 잡힌 전사는 건너뜀

        warrior_x, warrior_y = warrior_positions[i]

        # 시야 내에 있는 전사는 이동하지 않음
        if sight_map[warrior_x][warrior_y]:
            continue

        # 플레이어와의 맨해튼 거리 계산
        current_distance = moving_distance((player_x, player_y), (warrior_x, warrior_y))
        has_moved = False  # 이동 여부 플래그

        # 첫 번째 이동: 거리를 줄이기 위한 이동
        for d in range(4):
            next_x = warrior_x + move_dx[d]
            next_y = warrior_y + move_dy[d]

            # 이동할 위치가 그리드 내에 있고, 시야에 속하지 않는지 확인
            if next_x < 0 or next_y < 0 or next_x >= N or next_y >= N:
                continue
            if sight_map[next_x][next_y]:
                continue

            new_distance = moving_distance((player_x, player_y), (next_x, next_y))
            if new_distance < current_distance:
                warrior_x, warrior_y = next_x, next_y
                has_moved = True
                total_moved += 1
                break  # 첫 번째 이동 후 루프 탈출

        # 두 번째 이동: 추가로 거리를 줄일 수 있는지 확인
        if has_moved:
            new_distance = moving_distance((player_x, player_y), (warrior_x, warrior_y))
            for d in range(4):
                # d의 반대 방향 계산
                opposite_dir = (d + 2) % 4
                next_x = warrior_x + move_dx[opposite_dir]
                next_y = warrior_y + move_dy[opposite_dir]

                if next_x < 0 or next_y < 0 or next_x >= N or next_y >= N:
                    continue
                if sight_map[next_x][next_y]:
                    continue

                further_distance = moving_distance((player_x, player_y), (next_x, next_y))
                if further_distance < new_distance:
                    warrior_x, warrior_y = next_x, next_y
                    total_moved += 1
                    break  # 두 번째 이동 후 루프 탈출

        # 전사의 위치 업데이트
        warrior_positions[i] = (warrior_x, warrior_y)

    # 플레이어에게 도달한 전사 수 계산
    for i in range(M):
        if warrior_positions[i][0] == -1:
            continue  # 이미 잡힌 전사는 건너뜀
        if warrior_positions[i][0] == player_x and warrior_positions[i][1] == player_y:
            total_hits += 1
            warrior_positions[i] = (-1, -1)  # 전사를 잡힌 상태로 표시

    return total_moved, total_hits

def update_warrior_count_grid(N: int, M: int, warrior_positions: List[point]) -> List[List[int]]:

    warrior = [[0 for _ in range(N)] for _ in range(N)]
    for i in range(M):
        if warrior_positions[i][0] == -1:
            continue
        x, y = warrior_positions[i]
        warrior[x][y] += 1
    return warrior

# 빠른 입출력을 위해 sys.stdin 사용
input = sys.stdin.read
data = input().split()
idx = 0

N = int(data[idx]); idx += 1  # 그리드 크기
M = int(data[idx]); idx += 1  # 전사 수

ini_x = int(data[idx]); idx += 1
ini_y = int(data[idx]); idx += 1
end_x = int(data[idx]); idx += 1
end_y = int(data[idx]); idx += 1

warrior_positions = []
for _ in range(M):
    x = int(data[idx]); idx += 1
    y = int(data[idx]); idx += 1
    warrior_positions.append((x, y))

obstacle = []
for _ in range(N):
    row = []
    for _ in range(N):
        cell = int(data[idx]); idx += 1
        row.append(cell)
    obstacle.append(row)

assert obstacle[ini_x][ini_y] == 0, "시작 지점에 장애물이 있습니다."
assert obstacle[end_x][end_y] == 0, "종료 지점에 장애물이 있습니다."

distance= cal_distance(end_x, end_y, N, obstacle)
if distance[ini_x][ini_y] == -1:
    print("-1")
    sys.exit()

current_x, current_y = ini_x, ini_y
sight_map = [[0 for _ in range(N)] for _ in range(N)]
warrior = update_warrior_count_grid(N, M, warrior_positions)

while True:
    moved = False
    for d in range(4):
        next_x = current_x + x_direction[d]
        next_y = current_y + y_direction[d]
        if next_x < 0 or next_y < 0 or next_x >= N or next_y >= N:
            continue
        if distance[next_x][next_y] < distance[current_x][current_y]:
            current_x, current_y = next_x, next_y
            moved = True
            break

    if current_x == end_x and current_y == end_y:
        print("0")
        break

    for i in range(M):
        if warrior_positions[i][0] == current_x and warrior_positions[i][1] == current_y:
            warrior_positions[i] = (-1, -1)

    warrior = update_warrior_count_grid(N, M, warrior_positions)
    sight_coverage = choose_best_sight(current_x, current_y, N, warrior, sight_map)
    warriors_moved, warriors_hit = move_warriors(current_x, current_y, N, M, warrior_positions, sight_map)
    warrior = update_warrior_count_grid(N, M, warrior_positions)
    print(f"{warriors_moved} {sight_coverage} {warriors_hit}")


IndexError: list index out of range